In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint


In [2]:
url = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/driver_info.csv'
drivers = pd.read_csv(url)  # driver_info - информация про водителей
drivers.head()

,age,user_rating,user_rides,user_time_accident,user_id,sex,first_ride_date
0,27,9.0,865,19.0,l17437965W,1,2019-4-2
1,46,7.9,2116,11.0,Z12362316j,0,2021-11-19
2,59,7.8,947,4.0,g11098715c,0,2021-1-15
3,37,7.0,18,4.0,U12618125q,0,2019-11-20
4,39,8.2,428,7.0,A14375829B,0,2019-7-23


In [3]:
print(drivers['age'].median())
print(drivers['user_rating'].min())
print(drivers.shape)
print(drivers['sex'].value_counts())
drivers['user_time_accident'].value_counts()

33.0
5.8
(15153, 7)
sex
0    7577
1    7576
Name: count, dtype: int64


user_time_accident
6.0     843
3.0     794
8.0     777
4.0     760
11.0    752
       ... 
82.0      1
85.0      1
84.0      1
80.0      1
83.0      1
Name: count, Length: 84, dtype: int64

In [4]:
df1 = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2.2.6_df1.csv', index_col=0)
df2 = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2.2.6_df2.csv', index_col=0)

In [5]:
df = pd.concat((df1,df2))
df = df.query('rating > 4 & (stop_times == 2 | stop_times == 3 | stop_times == 5)')
user_white_list = ['g10995906g', 'H90312511Q', 'C27138482L', 'H22518078t', 'n17438464n', 'F10525768e', 's15596151o', 'i18954748S', 'E84048392U', 'E69893354s']
car_black_list = ['l-4285389z', 'n79502261g', 'N-1839822D']
mask1 = df['user_id'].isin(user_white_list)
mask2 = df['car_id'].isin(car_black_list)
df = df[~mask2]
df = df[mask1]
result = df[df.index % 2 == 0]
result

,user_id,car_id,rating,stop_times
20,C27138482L,M-1786506l,7.42,2
190,F10525768e,I98930031r,4.21,2
248,s15596151o,u-2070489i,5.39,2


In [6]:
drivers.query('(user_time_accident > 4.5 & user_time_accident <= 9) & age < 40').nunique()

age                     22
user_rating             37
user_rides            1025
user_time_accident       5
user_id               2478
sex                      2
first_ride_date       1073
dtype: int64

In [7]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2_2_8.csv')
df.shape

(100, 5)

In [8]:
df['tripple_accident'] = df['user_time_accident'] * 3
df['rides_on_training'] = df['user_rides'] / df['user_rating']
df1 = df.query('user_time_accident > 25')
result = df1.query('age > @df1.age.mean()')



In [9]:
df['tripple_accident'] = df['user_time_accident'] * 3
df['rides_on_training'] = df['user_rides'] / df['user_rating']
df1 = df.query('user_time_accident > 25')
result = df1.query('age > @df1.age.mean()')
result

,user_id,user_rating,age,user_rides,user_time_accident,tripple_accident,rides_on_training
3,E10711310m,8.4,33,322,39.0,117.0,38.333333
64,t25217963K,8.0,30,101,72.0,216.0,12.625000


In [10]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2_2_10.csv')
new_index = 50
new_data = ['new_car_id', 'new_worker_id', '2019-11-11 4:44','refuel_check', 3.5, 10]
del_index_list = [41, 18, 33, 7, 46]
index = 22
df.drop(del_index_list, axis=0, inplace=True)
df.loc[new_index,:] = new_data
df.loc[index,['destroy_degree']] += 1
df.drop('fix_date', axis=1, inplace=True)
print(df.shape)
df['efficient'] = df['destroy_degree'] / df['work_duration']
df['work_duration'] = df['work_duration']**2
df.rename(columns={'work_duration': 'work_duration_square'}, inplace=True)
print(df.shape)
df.drop_duplicates(subset='car_id', inplace=True)
print(df.shape)
df['work_duration_square'] = df['work_duration_square'].astype('int64')
df.tail()

(46, 5)
(46, 6)
(45, 6)


,car_id,worker_id,work_type,destroy_degree,work_duration_square,efficient
45,j-1549044A,ZF,refuel,5.0,2209,0.106383
47,B18597537S,NR,reparking,1.0,676,0.038462
48,H29930196K,NF,repair,5.0,225,0.333333
49,p-2255017N,MN,repair,7.7,1764,0.183333
50,new_car_id,new_worker_id,refuel_check,3.5,100,0.350000


In [33]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2.2.11.csv')
sort_by_list = ['car_rating', 'riders']
cols2drop = ['random_name_1', 'target_reg', 'target_class', 'random_name_2']
columns_order_list = ['model', 'car_type', 'car_id', 'car_rating', 'year_to_start', 'year_to_work', 'riders', 'fuel_type']
critical_car_rating = 2
df.sort_values(by=sort_by_list[0], inplace=True, ascending=False)
new_drops = [i for i in cols2drop if i in df.columns]
df.drop(new_drops, axis=1, inplace=True)
df = df.reindex(columns=columns_order_list)
df.query('car_rating >= @critical_car_rating',inplace=True)
df.query('(riders <= 10000) & (car_type != "standart" & car_type != "economy")', inplace=True)
df.head()

,model,car_type,car_id,car_rating,year_to_start,year_to_work,riders,fuel_type
4,Mini Cooper,business,i-4390733F,4.96,2012,2018,1007,petrol
295,Mini Cooper,business,b34607341j,4.04,2011,2018,2740,petrol


In [31]:
ans = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2_2_11_ans.csv', index_col=0)
ans

,model,car_type,car_id,car_rating,year_to_start,year_to_work,riders,fuel_type
4,Mini Cooper,business,i-4390733F,4.96,2012,2018,1007,petrol
295,Mini Cooper,business,b34607341j,4.04,2011,2018,2740,petrol


In [38]:
import pandas as pd

df = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2_2_12.csv')
mean_rating = df['car_rating'].mean()
df['car_rating'] += 1
df['year_to_work'] += 1
df.query('car_rating > @mean_rating', inplace=True)
df['year_to_work'] = df['year_to_work'].apply(lambda x: x + 2 if x < 2017 else int(x/5))
df.rename(columns={'year_to_start': 'new_year_to_start'},inplace=True)
df.drop(columns=['fuel_type'],inplace=True)
df['car_type'] = df['car_type'].apply(lambda x: 'super_' + str(x))
result = df

,Unnamed: 0,car_id,model,car_type,car_rating,new_year_to_start,riders,year_to_work,target_reg,target_class
0,0,o-1588203t,VW Polo VI,super_economy,5.04,2014,61872,2018,46.175419,engine_check
1,1,B-1760099A,Renault Kaptur,super_standart,5.36,2015,95768,404,30.585652,engine_overheat
2,2,u-2045579o,Renault Kaptur,super_standart,6.94,2012,21114,404,68.091881,engine_ignition
3,3,V-4730687q,VW Tiguan,super_economy,5.10,2016,95799,403,39.808397,break_bug
4,4,K23027223e,Smart ForTwo,super_economy,4.82,2017,127971,404,37.024779,electro_bug


In [39]:
df = pd.read_csv('https://stepik.org/media/attachments/lesson/681964/2_2_13.csv')

cols2drop = ['ride_cost', 'stop_times', 'user_ride_quality']
df['flag'] = df.apply(lambda x: 'good_line' if x['rating'] > 5 and x['speed_max'] < 100 else
("bad_line" if x['rating'] > 5 and x['speed_max'] > 100 else 'undefined'), axis=1)
df['speed_check'] = df.apply(lambda x: 'avg_more' if x['distance'] / x['ride_duration'] > x['speed_avg'] else 'avg_less', axis=1)
df.drop(columns=cols2drop, inplace=True)

df.dropna(subset=['car_id'], inplace=True)

result = df